In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install catboost

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import re
import math
import datetime
import random
import copy
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
#import train and test datasets
train1 = pd.read_csv('/home/felix/Downloads/zimnat/Train.csv', parse_dates = ['join_date'])
test1 = pd.read_csv('/home/felix/Downloads/zimnat/Test.csv', parse_dates=['join_date'])
submission = pd.read_csv('/home/felix/Downloads/zimnat/SampleSubmission.csv')

In [3]:
print(train1.shape, test1.shape, submission.shape)
print('---------------------------------------')
print(train1.columns)
print('---------------------------------------')
print(train1.dtypes)
print('---------------------------------------')
print(train1.isnull().sum().max(), test1.isnull().sum().max())

(29132, 29) (10000, 29) (210000, 2)
---------------------------------------
Index(['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code', 'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'], dtype='object')
---------------------------------------
ID                                  object
join_date                   datetime64[ns]
sex                                 object
marital_status                      object
birth_year                           int64
branch_code                         object
occupation_code                     object
occupation_category_code            object
P5DA                                 int64
RIBP                                 int64
8NN1                                 int64
7POT                                 int64
66FJ                                 int64
GYSR         

In [4]:
train1.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [5]:
test1.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [6]:
# Split features and labels
train1_labels = train1[['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']].reset_index(drop=True)
train1_features = train1.drop(columns=['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'], axis=1)
test1_features = test1.drop(columns=['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'], axis=1)
test1_labels = test1[['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']].reset_index(drop=True)

# Combine train and test features in order to apply the feature transformation pipeline to the entire dataset
all_features = pd.concat([train1_features, test1_features]).reset_index(drop=True)
print(all_features.shape, train1_labels.shape)

(39132, 8) (29132, 21)


In [7]:
# determine the threshold for missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:9]

Percent of missing data


[('join_date', 0.01),
 ('ID', 0.0),
 ('sex', 0.0),
 ('marital_status', 0.0),
 ('birth_year', 0.0),
 ('branch_code', 0.0),
 ('occupation_code', 0.0),
 ('occupation_category_code', 0.0)]

In [8]:
def handl_miss(df):
    df['join_date'] = df.groupby('branch_code')['join_date'].transform(lambda x: x.fillna(x.mode()[0]))
    return df


all_features = handl_miss(all_features)    

In [9]:
# Let's make sure we handled all the missing values
missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('ID', 0.0),
 ('join_date', 0.0),
 ('sex', 0.0),
 ('marital_status', 0.0),
 ('birth_year', 0.0),
 ('branch_code', 0.0),
 ('occupation_code', 0.0),
 ('occupation_category_code', 0.0)]

In [10]:
all_features.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS


In [11]:
all_features['age'] = 2020 - all_features['birth_year']
#train['join_year'] = train['join_date'].dt.year
#train['join_month'] = train['join_date'].dt.month
#train['join_day'] = train['join_date'].dt.day
dt = pd.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
all_features['time_period'] = dt - all_features['join_date']
all_features['ageWJ'] = all_features['join_date'].dt.year - all_features['birth_year']

In [12]:
all_features.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,age,time_period,ageWJ
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,33,595 days,32
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,39,591 days,38
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,29,2782 days,22
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,30,589 days,29
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,30,589 days,29


In [13]:
def logs(res, ls):
    m =res.shape[1]
    for l in ls:
        res = res.assign(newcol = pd.Series(np.log(1.01+res[l])).values)
        res.columns.values[m] = l + '_log'
        m += 1
    return res

log_features = ['birth_year', 'age', 'ageWJ']

all_features = logs(all_features, log_features)

In [14]:
def squares(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(res[l]*res[l]).values)
        res.columns.values[m] = l + '_sq'
        m += 1
    return res

squared_features = ['birth_year', 'age', 'ageWJ']
all_features = squares(all_features, squared_features)

In [15]:
all_features.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,age,time_period,ageWJ,birth_year_log,age_log,ageWJ_log,birth_year_sq,age_sq,ageWJ_sq
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,33,595 days,32,7.594889,3.526655,3.496811,3948169,1089,1024
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,39,591 days,38,7.591867,3.689129,3.663818,3924361,1521,1444
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,29,2782 days,22,7.596899,3.401531,3.135929,3964081,841,484
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,30,589 days,29,7.596397,3.434310,3.401531,3960100,900,841
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,30,589 days,29,7.596397,3.434310,3.401531,3960100,900,841


In [16]:
all_features = all_features.loc[:, ~all_features.columns.duplicated()]

In [17]:
X_train__ = all_features.iloc[:len(train1_labels), :]
X_test__ = all_features.iloc[len(train1_labels):, :]
X_train__.shape, train1_labels.shape, X_test__.shape, test1_labels.shape

((29132, 17), (29132, 21), (10000, 17), (10000, 21))

In [18]:
train = pd.concat([X_train__, train1_labels], axis=1)

In [19]:
X_test__ = X_test__.reset_index(drop=True)
test = pd.concat([X_test__, test1_labels], axis=1)

In [29]:
print(train.shape, test.shape)

(29132, 38) (10000, 38)


In [ ]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
    info = v[:17]
    binary = v[17:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
        c+=1
        for k == i:
            binary_transformed = list(copy.copy(binary))
            binary_transformed[i] = 0
            X_train.append(list(info) + binary_transformed + [X_train_columns[17+k]] + [c])
            
X_train = pd.DataFrame(X_train)
X_train.columns = []

In [ ]:
X_test = []
true_values = []
c = 0
for v in test.values:
    c =+ 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i ==1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = []

In [ ]:
features_train = []
features_test = []
columns = []

append_features = []

for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))
    
y_train = X_train[['product_pred']]

In [ ]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

In [ ]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [ ]:
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
  data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']